## Bloque 1 - Configuración Inicial

In [4]:
import glob
import os

from datetime import datetime
from pathlib import Path

import pandas as pd
import numpy as np
import sys
import json

In [5]:
#Parámetros de validación
pattern = "fundicion_ilo_*.pkl"
max_neg_net = 0.5
report_name = None
fail_on_error = False

In [6]:
#Rutas
BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR.parent/"data_sintetica"
OUT_DIR = BASE_DIR/"resultado de modelos"
OUT_DIR.mkdir(exist_ok=True, parents=True)

In [7]:
#Busca el PKL más reciente
pkls = sorted(glob.glob(str(DATA_DIR/pattern)))
if not pkls:
    raise FileNotFoundError(f"No se encontró ningún PKL con patrón '{pattern}' en {DATA_DIR}")

PKL_PATH = Path(pkls[-1])
print(f"PKL encontrado: {PKL_PATH}")

PKL encontrado: d:\IC\Y03AN1_Machine_Learning_I\data_sintetica\fundicion_ilo_20250814_2015.pkl


## Bloque 2 - Configuración de validación

In [8]:
# Columnas esperadas
ESPERADO = ["id_lote","fecha_llegada","proveedor","puerto_origen","mineral","metodo_transporte",
    "peso_tn","ley_cu_pct","humedad_pct","recuperacion_pct","precio_lme_usd_lb",
    "tc_usd_tn","costo_logistico_usd_tn","penalidad_as_usd_tn","ingreso_neto_usd"]

In [9]:
#Rangos aceptables
RANGOS = {
    'ley_cu_pct': (18, 36),
    'humedad_pct': (6, 11.5),
    'recuperacion_pct': (93, 98.5),
    'precio_lme_usd_lb': (3.2, 5.2),
    'tc_usd_tn': (70, 130),
    'costo_logistico_usd_tn': (18, 48),
    'penalidad_as_usd_tn': (0, 45),
    'peso_tn': (8000, 40000),
}

In [11]:
#Lista de columnas por tipo
NUM_COLS = ['peso_tn','ley_cu_pct','humedad_pct','recuperacion_pct','precio_lme_usd_lb',
            'tc_usd_tn','costo_logistico_usd_tn','penalidad_as_usd_tn','ingreso_neto_usd']

CAT_COLS = ['proveedor','puerto_origen','mineral','metodo_transporte']

In [12]:
#Función para detectar fuera de rango
def fuera_de_rango(s: pd.Series, lo: float, hi: float) -> pd.Series:
    return (s < lo) | (s > hi)

In [14]:
#Carga de datos
df = pd.read_pickle(PKL_PATH)
print(f"DataFrame cargado con {len(df):,} filas y {df.shape[1]} columnas")

if "fecha_llegada" in df.columns:
    df["fecha_llegada"] = pd.to_datetime(df["fecha_llegada"], errors = "coerce")

for c in NUM_COLS:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

df.head()

DataFrame cargado con 5,000 filas y 15 columnas


,id_lote,fecha_llegada,proveedor,puerto_origen,mineral,metodo_transporte,peso_tn,ley_cu_pct,humedad_pct,recuperacion_pct,precio_lme_usd_lb,tc_usd_tn,costo_logistico_usd_tn,penalidad_as_usd_tn,ingreso_neto_usd
0,L-202406-00001,2024-06-02,Proveedor_09,Paita,cobre,tren,22345.39,29.16,6.83,96.12,4.097,79.25,20.12,0.34,52352585.96
1,L-202505-00002,2025-05-01,Proveedor_15,Mejillones,cobre,tren,22004.23,35.53,7.96,97.08,3.865,97.07,31.32,3.28,59505579.17
2,L-202411-00003,2024-11-17,Proveedor_16,Arica,cobre,barco,8213.76,28.98,7.24,95.70,3.792,101.38,42.52,6.18,17145773.90
3,L-202406-00004,2024-06-06,Proveedor_10,Callao,cobre,barco,24789.30,26.83,10.80,95.46,4.072,106.95,29.77,10.64,51363864.59
4,L-202405-00005,2024-05-02,Proveedor_05,Paita,cobre,barco,15702.17,25.61,9.51,96.66,3.831,105.00,29.17,6.53,29476513.33


In [15]:
# Validación del esquema
errores = []
advertencias = []
checks = []

cols_ok = list(df.columns) == ESPERADO

if not cols_ok:
    diff_mas = list(set(df.columns) - set(ESPERADO))
    diff_menos = list(set(ESPERADO) - set(df.columns))
    errores.append({
        "tipo": "esquema",
        "mensaje": f"Columnas inesperadas: +{diff_mas} / faltantes: {diff_menos}"
    })

print("Errores de esquema", errores if errores else "Ninguno")

Errores de esquema Ninguno
